# 🚀 Analyse Rhétorique Collaborative par Agents IA - Exécuteur Principal

**Objectif:** Ce notebook orchestre et exécute une analyse rhétorique multi-agents sur un texte donné. Il sert de point d'entrée principal pour lancer le processus.

**Structure Modulaire:**
1.  `Argument_Analysis_UI_configuration.ipynb` : Gère l'interface utilisateur pour sélectionner/préparer le texte à analyser (incluant sources prédéfinies, URL, fichier, texte direct, et extraction) et charge/sauvegarde la configuration des sources.
2.  `Argument_Analysis_Agentic-0-init.ipynb`: Configuration initiale (dépendances, LLM, JVM), définition de l'état partagé (`RhetoricalAnalysisState`) et du gestionnaire d'état (`StateManagerPlugin`).
3.  `Argument_Analysis_Agentic-1-informal_agent.ipynb`: Définition de l'`InformalAnalysisAgent`.
4.  `Argument_Analysis_Agentic-2-pl_agent.ipynb`: Définition du `PropositionalLogicAgent`.
5.  `Argument_Analysis_Agentic-3-orchestration.ipynb`: Définition des stratégies d'orchestration et de la fonction principale `run_analysis_conversation`.

**Prérequis:**
* Un fichier `.env` à la racine contenant les clés API, configurations LLM, et la clé de chiffrement `TEXT_CONFIG_KEY`.
* Un environnement Java Development Kit (JDK >= 11) correctement installé et configuré (`JAVA_HOME`).
* Les dépendances Python installées (`ipywidgets`, `requests`, `jupyter-ui-poll`, `python-dotenv`, `semantic-kernel`, `pandas`, `jpype1`, `cryptography`).
* Les JARs Tweety placés dans le dossier `libs/`.
* Le fichier `extract_sources.json.gz.enc` (s'il existe déjà) contenant les définitions des sources.

## Présentation générale du système

### Objectifs pédagogiques

Ce notebook illustre une architecture d'**agents collaboratifs multi-modaux** pour l'analyse rhétorique automatisée. Les étudiants découvriront:

1. **Orchestration d'agents IA** (Semantic Kernel)
2. **Analyse hybride**: LLM (compréhension naturelle) + Logique formelle (preuve)
3. **État partagé** pour communication inter-agents
4. **Validation structurée** avec métriques de confiance

### Architecture du système

Le système combine deux paradigmes d'IA complémentaires:

| Composant | Technologie | Rôle |
|-----------|-------------|------|
| **InformalAnalysisAgent** | LLM (GPT-4/Claude) | Comprend langage naturel, identifie arguments/sophismes |
| **PropositionalLogicAgent** | Tweety (bibliothèque Java) | Formalise en logique PL, prouve cohérence/implications |
| **StateManager** | Dataclass Python | Mémoire partagée entre agents |
| **Orchestration** | Semantic Kernel | Coordination des appels d'agents |

### Workflow en 3 phases

```
Phase 1: ANALYSE INFORMELLE
├─ Détection arguments (prémisses, conclusions)
├─ Identification sophismes (taxonomie hiérarchique)
└─ Stockage état (JSON structuré)

Phase 2: FORMALISATION LOGIQUE
├─ Traduction propositions en PL
├─ Création Belief Sets (Tweety)
├─ Requêtes logiques (cohérence, implication)
└─ Stockage preuves formelles

Phase 3: VALIDATION & SYNTHÈSE
├─ Cross-validation (6 critères)
├─ Calcul score de confiance
├─ Génération conclusion synthétique
└─ Export rapport JSON
```

> **Cas d'usage réel**: Ce système peut analyser des débats politiques, détecter la manipulation rhétorique, ou servir d'outil pédagogique pour l'enseignement de la pensée critique.

## 1. Chargement de l'Environnement

Chargement des variables depuis le fichier `.env` (clés API, clé de chiffrement, etc.).

### Rôle du fichier .env

Le fichier `.env` centralise toutes les configurations sensibles et paramètres du système:

| Variable | Fonction |
|----------|----------|
| `OPENAI_API_KEY` | Authentification API OpenAI pour les LLMs |
| `ANTHROPIC_API_KEY` | Authentification API Anthropic (optionnel) |
| `TEXT_CONFIG_KEY` | Clé de chiffrement pour la config sécurisée |
| `BATCH_MODE` | Active le mode non-interactif pour tests automatisés |
| `BATCH_TEXT` | Texte personnalisé pour mode batch (optionnel) |

> **Note de sécurité**: Le fichier `.env` ne doit jamais être versionné avec Git. Utilisez `.env.example` comme template.

In [ ]:
# Charger les variables d'environnement
from dotenv import load_dotenv, find_dotenv
loaded_env = load_dotenv(find_dotenv(), override=True)
print(f".env chargé: {loaded_env}") # Affiche True si le .env a été trouvé et chargé

### Interprétation du résultat

**Sortie attendue**: `.env chargé: True`

Si vous obtenez `False`, cela signifie:
- Le fichier `.env` n'existe pas dans le répertoire courant ou ses parents
- Les variables d'environnement doivent être définies manuellement

**Actions correctives**:
1. Copier `.env.example` vers `.env`
2. Remplir les valeurs manquantes (clés API)
3. Relancer cette cellule

### Vérification des dépendances

Avant de continuer, assurez-vous que les dépendances suivantes sont satisfaites:

**Dépendances Python**:
```bash
pip install python-dotenv semantic-kernel pandas jpype1 cryptography
pip install ipywidgets requests jupyter-ui-poll  # Pour mode interactif
```

**Dépendances système**:
- **JDK 11+** (pour Tweety via JPype): `JAVA_HOME` doit être configuré
- **Bibliothèque Tweety**: JARs dans `libs/` (téléchargement automatique via `Tweety-1-Setup.ipynb`)

**Fichiers requis**:
| Fichier | Rôle | Création |
|---------|------|----------|
| `.env` | Variables d'environnement (clés API) | Copier depuis `.env.example` |
| `libs/*.jar` | Bibliothèque Tweety Java | Téléchargement auto |
| `data/logical_fallacies_taxonomy.json` | Taxonomie sophismes | Fourni |

> **Astuce**: Si JPype échoue à s'initialiser, le système basculera en mode "fallback" (sans logique formelle). L'analyse informelle fonctionnera toujours.

## 2. Chargement de l'Interface Utilisateur

Exécution du notebook `UI_Configuration.ipynb` pour définir la fonction `configure_analysis_task()`. C'est ce notebook qui contient désormais toute la logique de l'interface graphique, du cache fichier et de la gestion de la configuration chiffrée.

In [ ]:
# Mode batch contrôlé par variable d'environnement
# Si BATCH_MODE=true dans .env, on skip l'UI interactive (widgets bloquants)
import os
BATCH_MODE = os.getenv("BATCH_MODE", "false").lower() in ("true", "1", "yes")

if BATCH_MODE:
    print("Mode BATCH detecte (BATCH_MODE=true dans .env)")
    print("   -> Skip du chargement UI_configuration.ipynb (widgets non compatibles)")
    print("   -> Le texte sera fourni directement dans la cellule suivante")
else:
    # Exécuter le notebook UI pour définir la fonction configure_analysis_task
    # Assurez-vous que le fichier UI_Configuration.ipynb est dans le même répertoire
    print("Exécution de Argument_Analysis_UI_configuration.ipynb...")
    %run ./Argument_Analysis_UI_configuration.ipynb
    print("Exécution de Argument_Analysis_UI_configuration.ipynb terminée.")

    # Vérification que la fonction est bien définie après l'exécution
    if 'configure_analysis_task' not in locals():
        print("ERREUR CRITIQUE : La fonction configure_analysis_task n'a pas été définie par UI_Configuration.ipynb !")
    else:
        print("Fonction configure_analysis_task trouvée.")

### Modes d'exécution: Batch vs Interactif

Le notebook supporte deux modes d'exécution distincts:

| Mode | Activation | Usage | Interface |
|------|------------|-------|-----------|
| **Interactif** | `BATCH_MODE=false` (défaut) | Démonstration, exploration | Widgets Jupyter (ipywidgets) |
| **Batch** | `BATCH_MODE=true` dans `.env` | Tests automatisés, CI/CD, Papermill | Texte prédéfini sans interaction |

**Pourquoi le mode batch?**

Les widgets Jupyter (`ipywidgets`) utilisent des boucles de polling bloquantes incompatibles avec:
- L'exécution Papermill
- Les MCP (Model Context Protocol) Jupyter
- Les pipelines CI/CD automatisés

Le mode batch permet de contourner ces limitations en sautant le chargement de `UI_configuration.ipynb` et en utilisant directement un texte source.

> **Note technique**: En mode batch, le texte est fourni via `BATCH_TEXT` (variable d'environnement) ou via le texte d'exemple enrichi défini dans la cellule suivante.

## 3. Configuration de la Tâche et Récupération du Texte

### Sélection du texte à analyser

Cette étape configure le texte source selon le mode d'exécution:

**Mode Interactif**:
- Interface graphique avec widgets Jupyter
- Choix entre sources prédéfinies, URL, fichier, ou texte direct
- Extraction de contenu (HTML, PDF, etc.)
- Configuration chiffrée sauvegardée (`extract_sources.json.gz.enc`)

**Mode Batch**:
- Texte fourni via variable d'environnement `BATCH_TEXT`
- Ou texte d'exemple prédéfini (débat sur transition énergétique)
- Pas d'interaction utilisateur requise

> **Attention**: En mode interactif, la cellule suivante attendra que vous cliquiez sur "Lancer l'Analyse" dans l'interface. Ne passez pas à la suite avant d'avoir sélectionné un texte.

In [ ]:
# Configuration du texte pour l'analyse
# En mode BATCH : utilise BATCH_TEXT (variable d'env) ou le texte d'exemple
# En mode interactif : appelle configure_analysis_task() (si disponible)

texte_pour_analyse = None

# Texte d'exemple enrichi pour le mode batch - contient des arguments complexes et des sophismes
TEXTE_EXEMPLE_BATCH = """Le debat sur la transition energetique en France revele des enjeux majeurs pour notre avenir.

ARGUMENTS EN FAVEUR DES ENERGIES RENOUVELABLES:
Les energies renouvelables sont essentielles pour lutter contre le changement climatique. La France s'est engagee a reduire ses emissions de CO2 de 40% d'ici 2030, ce qui necessite un investissement massif dans le solaire et l'eolien. D'ailleurs, l'Allemagne a deja prouve que c'etait possible en produisant 46% de son electricite a partir de sources renouvelables en 2023.

De plus, les energies renouvelables creent des emplois locaux non delocalisables. Selon l'ADEME, le secteur emploie deja plus de 150 000 personnes en France et pourrait en creer 500 000 supplementaires d'ici 2050. Quiconque s'oppose a cette transition est donc contre la creation d'emplois.

ARGUMENTS CONTRE LE DEVELOPPEMENT MASSIF DES RENOUVELABLES:
Cependant, les ecologistes veulent nous ramener a l'age de pierre en supprimant le nucleaire, qui fournit pourtant 70% de notre electricite. Sans le nucleaire, nous serions obliges de choisir entre des coupures d'electricite massives ou une dependance totale au gaz russe.

Le cout de la transition est exorbitant. Tout le monde sait que les eoliennes sont inefficaces car elles ne tournent que 25% du temps. Mon voisin Jean-Pierre, qui est ingenieur a EDF, m'a dit que les renouvelables ne pourront jamais remplacer le nucleaire. C'est un expert, donc il a forcemment raison.

De plus, les panneaux solaires sont fabriques en Chine avec du charbon, ce qui annule completement leur benefice ecologique. Donc investir dans le solaire revient a financer la pollution chinoise.

CONCLUSION:
Il faut arreter de debattre et agir. Soit on accepte une transition energetique complete vers 100% de renouvelables d'ici 2040, soit on condamne nos enfants a vivre sur une planete inhabitable. Des milliers de scientifiques ont signe une petition pour le climat, ce qui prouve definitivement que nous avons raison de promouvoir les energies vertes. Les climatosceptiques sont finances par les lobbies petroliers et ne meritent pas qu'on ecoute leurs arguments.

En conclusion, bien que certains arguments des deux cotes aient du merite, la question necessite une analyse nuancee des compromis entre independance energetique, impact environnemental, cout economique et faisabilite technique."""

if BATCH_MODE:
    # === MODE BATCH ===
    print("Mode BATCH - Configuration automatique du texte")
    
    # Priorite 1: Variable d'environnement BATCH_TEXT
    batch_text_env = os.getenv("BATCH_TEXT", "")
    if batch_text_env:
        texte_pour_analyse = batch_text_env
        print(f"Texte charge depuis BATCH_TEXT ({len(texte_pour_analyse)} caracteres)")
    else:
        # Priorite 2: Texte d'exemple par defaut
        texte_pour_analyse = TEXTE_EXEMPLE_BATCH
        print(f"Texte d'exemple enrichi utilise ({len(texte_pour_analyse)} caracteres)")
    
    print(f"\nExtrait du texte:\n{texte_pour_analyse[:200]}...")

else:
    # === MODE INTERACTIF ===
    print("Mode INTERACTIF - Lancement de l'interface de configuration")
    
    if 'configure_analysis_task' in locals():
        try:
            texte_pour_analyse = configure_analysis_task()
            print(f"Texte recupere via l'interface ({len(texte_pour_analyse) if texte_pour_analyse else 0} caracteres)")
        except Exception as e_ui:
            print(f"Erreur lors de la configuration UI : {e_ui}")
            import traceback
            traceback.print_exc()
    else:
        print("Fonction configure_analysis_task non disponible - verifiez le chargement de UI_configuration.ipynb")

# Verification finale
if not texte_pour_analyse:
    print("\nAucun texte prepare. L'analyse ne peut pas continuer.")
else:
    print(f"\nTexte pret pour l'analyse (longueur: {len(texte_pour_analyse)}). Passage au chargement des agents.")

### Interprétation de la configuration du texte

**Résultat attendu (Mode Batch)**:
```
Mode BATCH - Configuration automatique du texte
Texte charge depuis BATCH_TEXT (XXX caracteres)
ou
Texte d'exemple enrichi utilise (XXX caracteres)
```

**Résultat attendu (Mode Interactif)**:
```
Mode INTERACTIF - Lancement de l'interface de configuration
Texte recupere via l'interface (XXX caracteres)
```

**Structure du texte d'exemple**:

Le texte d'exemple enrichi contient intentionnellement plusieurs sophismes pour démonstration:

| Sophisme | Exemple dans le texte |
|----------|----------------------|
| Faux dilemme | "Soit 100% renouvelables, soit planète inhabitable" |
| Appel à l'autorité | "Mon voisin ingénieur EDF dit que..." |
| Ad hominem | "Climatosceptiques financés par lobbies pétroliers" |
| Généralisation hâtive | "Tout le monde sait que les éoliennes..." |
| Pente glissante | "Les écologistes veulent nous ramener à l'âge de pierre" |

> **Conseil pédagogique**: Le texte d'exemple est conçu pour tester la robustesse de l'analyse rhétorique multi-agents. Utilisez-le pour comprendre comment les agents détectent les arguments fallacieux.

## 4. Chargement des Définitions des Agents et de l'Orchestration

### Architecture modulaire par notebooks

Le système est structuré en **4 notebooks enfants** chargés dynamiquement via `%run`:

**Avantages de cette architecture**:
1. **Séparation des responsabilités**: Chaque agent dans son propre fichier
2. **Réutilisabilité**: Les agents peuvent être chargés individuellement
3. **Maintenabilité**: Modification d'un agent sans toucher aux autres
4. **Testabilité**: Tests unitaires par agent

**Ordre de chargement critique**:

```
0-init.ipynb           (État + Config LLM + JVM)
    ↓
1-informal_agent.ipynb (Dépend de RhetoricalAnalysisState)
    ↓
2-pl_agent.ipynb       (Dépend de RhetoricalAnalysisState + JVM)
    ↓
3-orchestration.ipynb  (Dépend de tous les agents)
```

> **Note technique**: `%run` exécute le notebook dans le namespace courant. Toutes les variables définies dans les notebooks enfants (classes, fonctions) deviennent disponibles ici.

In [ ]:
# Exécuter les notebooks enfants pour charger les définitions
# Seulement si un texte a été préparé avec succès
if 'texte_pour_analyse' in locals() and texte_pour_analyse:
    print("\nChargement des définitions des agents et de l'orchestration...")
    try:
        %run ./Argument_Analysis_Agentic-0-init.ipynb
        %run ./Argument_Analysis_Agentic-1-informal_agent.ipynb
        %run ./Argument_Analysis_Agentic-2-pl_agent.ipynb
        %run ./Argument_Analysis_Agentic-3-orchestration.ipynb  # Définit run_analysis_conversation(texte_a_analyser)
        print("✅ Définitions chargées.")
        # Vérifier que la fonction d'orchestration est chargée
        if 'run_analysis_conversation' not in locals():
             print("❌ ERREUR CRITIQUE: La fonction run_analysis_conversation n'a pas été définie après l'exécution des notebooks agents!")
             # raise NameError("run_analysis_conversation non définie")
    except Exception as e_run:
        print(f"\n❌ Une erreur est survenue lors de l'exécution des notebooks enfants : {e_run}")
        import traceback
        traceback.print_exc()
        # Empêcher la suite si le chargement échoue
        texte_pour_analyse = None
else:
    print("\nSkipping agent definition loading because no text was prepared.")

### Architecture modulaire chargée

**Notebooks chargés et leurs responsabilités**:

| Notebook | Composants définis | Rôle |
|----------|-------------------|------|
| `Agentic-0-init.ipynb` | `RhetoricalAnalysisState`, `StateManagerPlugin`, kernel SK | État partagé, configuration LLM, JVM Tweety |
| `Agentic-1-informal_agent.ipynb` | `InformalAnalysisAgent` | Analyse arguments et sophismes (taxonomie) |
| `Agentic-2-pl_agent.ipynb` | `PropositionalLogicAgent`, `PropositionalLogicPlugin` | Formalisation en logique propositionnelle |
| `Agentic-3-orchestration.ipynb` | `run_analysis_conversation()` | Orchestration collaborative des agents |

**Architecture technique**:

```
┌─────────────────────────────────────────────────┐
│  Semantic Kernel (orchestration)                │
│  ┌──────────────┐  ┌────────────────────────┐  │
│  │ Informal     │  │ Propositional Logic    │  │
│  │ Analysis     │─▶│ Agent (Tweety/JPype)   │  │
│  │ Agent        │  │                        │  │
│  └──────────────┘  └────────────────────────┘  │
│         │                     │                 │
│         └─────────┬───────────┘                 │
│                   ▼                             │
│        ┌────────────────────┐                   │
│        │ StateManager       │                   │
│        │ Plugin             │                   │
│        │ (état partagé)     │                   │
│        └────────────────────┘                   │
└─────────────────────────────────────────────────┘
```

> **Note d'implémentation**: L'état partagé (`RhetoricalAnalysisState`) permet aux agents de communiquer leurs résultats via un contexte commun plutôt que par messages directs.

## 5. Exécution de l'Analyse Collaborative

### Orchestration asynchrone avec Semantic Kernel

Cette cellule lance le processus d'analyse multi-agents. L'orchestration fonctionne en **mode asynchrone** pour gérer efficacement les appels API LLM.

**Composants techniques**:

| Composant | Rôle |
|-----------|------|
| `nest_asyncio.apply()` | Permet `await` dans Jupyter (normalement synchrone) |
| `run_analysis_conversation(texte)` | Fonction d'orchestration principale |
| `local_state` | Objet `RhetoricalAnalysisState` retourné |

**Étapes d'exécution internes**:

1. **Initialisation**: Création kernel SK, chargement plugins
2. **Tour 1 - Analyse informelle**: Agent LLM identifie arguments/sophismes
3. **Tour 2 - Formalisation**: Agent PL traduit en logique, exécute requêtes
4. **Synthèse**: Génération conclusion basée sur analyse hybride
5. **Retour**: État final JSON avec tous les résultats

> **Note performance**: L'exécution prend 90-150s selon la complexité du texte et la latence des API OpenAI/Anthropic. Surveillez les logs pour suivre la progression.

In [ ]:
import nest_asyncio
import asyncio

# Lancer seulement si on a obtenu un texte valide ET que les définitions sont chargées
if 'texte_pour_analyse' in locals() and texte_pour_analyse and 'run_analysis_conversation' in locals():
    print("\n🚀 Lancement de l'exécution asynchrone de l'analyse...")
    nest_asyncio.apply()
    try:
        # Passer le texte préparé
        local_state = await run_analysis_conversation(texte_pour_analyse)
        print("\n🏁 Analyse terminée.")
    except Exception as e_analysis:
        print(f"\n❌ Une erreur est survenue pendant l'exécution de l'analyse : {e_analysis}")
        import traceback
        traceback.print_exc()

elif 'texte_pour_analyse' not in locals() or not texte_pour_analyse:
    print("\n Analyse non lancée : aucun texte n'a été préparé ou une erreur est survenue avant.")
else: # Implique que run_analysis_conversation n'a pas été chargée
     print("\n Analyse non lancée : la fonction d'orchestration n'a pas pu être chargée.")

### Interprétation du processus d'analyse

**Étapes d'exécution observables**:

1. **Analyse informelle** (InformalAnalysisAgent):
   - Identification des arguments principaux
   - Détection des sophismes via taxonomie
   - Stockage dans `local_state.identified_arguments` et `identified_fallacies`

2. **Formalisation logique** (PropositionalLogicAgent):
   - Traduction des arguments en propositions PL
   - Création de Belief Sets Tweety
   - Exécution de requêtes logiques (cohérence, implication)

3. **Synthèse collaborative**:
   - Combinaison des résultats informels et formels
   - Génération de la conclusion finale
   - Stockage dans `local_state.final_conclusion`

**Durée typique**: 90-150 secondes (selon complexité du texte et latence API)

**Logs à surveiller**:

| Message | Signification |
|---------|---------------|
| `[InformalAnalysisAgent] Processing...` | Analyse rhétorique en cours |
| `[PropositionalLogicAgent] Creating belief set...` | Formalisation PL |
| `FUNC_ERROR: JPype not initialized` | Tweety non disponible (fallback mode) |
| `Final state:` | Affichage JSON de l'état final |

> **Note de performance**: En mode batch, l'exécution complète prend environ 122 secondes sur le texte d'exemple enrichi.

## 5bis. Rapport de Validation de l'Analyse

### Validation structurée avec métriques de confiance

Cette section génère un **rapport de validation cross-validé** qui vérifie la complétude et la cohérence de l'analyse rhétorique.

**Objectif pédagogique**: Démontrer l'importance de la validation automatisée dans les systèmes d'IA, en particulier pour:
- Détecter les analyses incomplètes
- Calculer un score de confiance quantitatif
- Identifier les composants manquants
- Fournir un rapport exploitable (JSON)

**Méthode de validation**:

La validation cross-check vérifie 6 critères indépendants:

| Critère | Validation | Poids |
|---------|------------|-------|
| `ARGUMENTS_IDENTIFIED` | `len(identified_arguments) > 0` | 1/6 |
| `FALLACIES_ANALYZED` | `len(identified_fallacies) > 0` | 1/6 |
| `BELIEF_SET_CREATED` | `len(belief_sets) > 0` | 1/6 |
| `QUERIES_EXECUTED` | `len(query_log) > 0` | 1/6 |
| `QUERIES_MEANINGFUL` | Au moins 1 résultat ACCEPTED/REJECTED | 1/6 |
| `CONCLUSION_GENERATED` | `final_conclusion is not None` | 1/6 |

**Score de confiance**: `confidence = nombre_critères_validés / 6`

> **Application pratique**: Ce type de validation est essentiel dans les systèmes de production pour garantir la qualité des analyses automatisées. Le rapport JSON peut alimenter des tableaux de bord de monitoring.

In [ ]:
# === CELLULE DE VALIDATION FINALE ===
# Genere un rapport JSON structure avec cross-validation

import json
from datetime import datetime
from typing import Dict, Any, List, Optional

def generate_validated_analysis_report(state) -> Dict[str, Any]:
    """Genere rapport JSON structure avec cross-validation."""
    
    report = {
        "metadata": {
            "timestamp": datetime.now().isoformat(),
            "version": "2.0-validated",
            "text_length": len(state.raw_text) if state.raw_text else 0,
            "text_snippet": (state.raw_text[:150] + "...") if state.raw_text and len(state.raw_text) > 150 else (state.raw_text or "")
        },
        "informal_analysis": {
            "arguments": [],
            "fallacies": [],
            "taxonomy_families_used": set()
        },
        "formal_analysis": {
            "belief_sets": [],
            "query_results": [],
            "consistency_checked": False
        },
        "cross_validation": {
            "validation_status": "INCOMPLETE",
            "confidence_score": 0.0,
            "checks_passed": [],
            "issues": []
        },
        "conclusion": {
            "summary": state.final_conclusion if hasattr(state, 'final_conclusion') else None,
            "is_complete": hasattr(state, 'final_conclusion') and state.final_conclusion is not None
        }
    }
    
    # Populate arguments
    if hasattr(state, 'identified_arguments'):
        for arg_id, arg_desc in state.identified_arguments.items():
            has_fallacy = False
            if hasattr(state, 'identified_fallacies'):
                has_fallacy = any(
                    f.get('target_argument_id') == arg_id
                    for f in state.identified_fallacies.values()
                )
            report["informal_analysis"]["arguments"].append({
                "id": arg_id, "description": str(arg_desc)[:200], "has_fallacy": has_fallacy
            })
    
    # Populate fallacies
    if hasattr(state, 'identified_fallacies'):
        for f_id, f_data in state.identified_fallacies.items():
            fallacy_type = f_data.get('type', 'Unknown') if isinstance(f_data, dict) else str(f_data)
            report["informal_analysis"]["fallacies"].append({
                "id": f_id,
                "type": fallacy_type,
                "justification": f_data.get('justification', '') if isinstance(f_data, dict) else '',
                "target_id": f_data.get('target_argument_id') if isinstance(f_data, dict) else None,
                "severity": "HIGH" if any(kw in str(fallacy_type).lower() for kw in ['manipulation', 'tromperie']) else "MEDIUM"
            })
            # Track taxonomy families
            if isinstance(fallacy_type, str) and '/' in fallacy_type:
                report["informal_analysis"]["taxonomy_families_used"].add(fallacy_type.split('/')[0])
    
    report["informal_analysis"]["taxonomy_families_used"] = list(report["informal_analysis"]["taxonomy_families_used"])
    
    # Populate belief sets
    if hasattr(state, 'belief_sets'):
        for bs_id, bs_data in state.belief_sets.items():
            content = bs_data.get('content', '') if isinstance(bs_data, dict) else str(bs_data)
            report["formal_analysis"]["belief_sets"].append({
                "id": bs_id,
                "logic_type": bs_data.get('logic_type', 'PL') if isinstance(bs_data, dict) else 'PL',
                "formula_count": content.count('\n') + 1 if content else 0,
                "is_consistent": "NOT_CHECKED"
            })
    
    # Populate query results
    if hasattr(state, 'query_log'):
        for qlog in state.query_log:
            raw_result = qlog.get('raw_result', '') if isinstance(qlog, dict) else ''
            status = "UNKNOWN"
            if "ACCEPTED" in str(raw_result): status = "ACCEPTED"
            elif "REJECTED" in str(raw_result): status = "REJECTED"
            elif "FUNC_ERROR" in str(raw_result): status = "ERROR"
            
            report["formal_analysis"]["query_results"].append({
                "log_id": qlog.get('log_id', '') if isinstance(qlog, dict) else '',
                "belief_set_id": qlog.get('belief_set_id', '') if isinstance(qlog, dict) else '',
                "query": qlog.get('query', '') if isinstance(qlog, dict) else '',
                "status": status
            })
    
    # === CROSS-VALIDATION LOGIC ===
    checks = []
    issues = []
    
    # Check 1: Arguments identified
    if len(report["informal_analysis"]["arguments"]) > 0:
        checks.append("ARGUMENTS_IDENTIFIED")
    else:
        issues.append("Aucun argument identifie")
    
    # Check 2: Fallacy analysis attempted
    if len(report["informal_analysis"]["fallacies"]) > 0:
        checks.append("FALLACIES_ANALYZED")
    elif hasattr(state, 'answers') and any("sophisme" in str(v).lower() for v in state.answers.values()):
        checks.append("FALLACY_ANALYSIS_ATTEMPTED")
    else:
        issues.append("Analyse sophismes non effectuee")
    
    # Check 3: Formal logic translation
    if len(report["formal_analysis"]["belief_sets"]) > 0:
        checks.append("BELIEF_SET_CREATED")
    else:
        issues.append("Aucun Belief Set PL cree")
    
    # Check 4: Queries executed
    if len(report["formal_analysis"]["query_results"]) > 0:
        checks.append("QUERIES_EXECUTED")
        accepted = sum(1 for q in report["formal_analysis"]["query_results"] if q["status"] == "ACCEPTED")
        rejected = sum(1 for q in report["formal_analysis"]["query_results"] if q["status"] == "REJECTED")
        if accepted > 0 or rejected > 0:
            checks.append("QUERIES_MEANINGFUL")
    else:
        issues.append("Aucune requete PL executee")
    
    # Check 5: Conclusion generated
    if report["conclusion"]["is_complete"]:
        checks.append("CONCLUSION_GENERATED")
    else:
        issues.append("Conclusion finale non generee")
    
    # Calculate confidence score
    max_checks = 6  # ARGUMENTS, FALLACIES, BELIEF_SET, QUERIES, QUERIES_MEANINGFUL, CONCLUSION
    confidence = len(checks) / max_checks
    report["cross_validation"]["confidence_score"] = round(confidence, 2)
    report["cross_validation"]["checks_passed"] = checks
    report["cross_validation"]["issues"] = issues
    
    # Determine validation status
    if confidence >= 0.8:
        report["cross_validation"]["validation_status"] = "COMPLETE_VALIDATED"
    elif confidence >= 0.5:
        report["cross_validation"]["validation_status"] = "PARTIAL_VALIDATED"
    elif confidence >= 0.3:
        report["cross_validation"]["validation_status"] = "MINIMAL"
    else:
        report["cross_validation"]["validation_status"] = "INCOMPLETE"
    
    return report


def display_validation_summary(report: Dict[str, Any]) -> str:
    """Affiche resume lisible du rapport de validation."""
    print("\n" + "="*70)
    print("          RAPPORT D'ANALYSE RHETORIQUE VALIDEE")
    print("="*70)
    
    cv = report["cross_validation"]
    status_symbols = {
        "COMPLETE_VALIDATED": "[OK]",
        "PARTIAL_VALIDATED": "[PARTIEL]",
        "MINIMAL": "[MINIMAL]",
        "INCOMPLETE": "[INCOMPLET]"
    }
    
    print(f"\n  STATUT: {status_symbols.get(cv['validation_status'], '?')} {cv['validation_status']}")
    print(f"  CONFIANCE: {cv['confidence_score']*100:.0f}%")
    
    print(f"\n  [ANALYSE INFORMELLE]")
    print(f"    Arguments: {len(report['informal_analysis']['arguments'])}")
    print(f"    Sophismes: {len(report['informal_analysis']['fallacies'])}")
    
    print(f"\n  [ANALYSE FORMELLE]")
    print(f"    Belief Sets: {len(report['formal_analysis']['belief_sets'])}")
    print(f"    Requetes: {len(report['formal_analysis']['query_results'])}")
    
    print(f"\n  [VALIDATIONS PASSEES]")
    for check in cv['checks_passed']:
        print(f"    [+] {check}")
    
    if cv['issues']:
        print(f"\n  [PROBLEMES DETECTES]")
        for issue in cv['issues']:
            print(f"    [-] {issue}")
    
    print(f"\n  [CONCLUSION]")
    if report['conclusion']['is_complete']:
        conclusion_preview = str(report['conclusion']['summary'])[:200]
        print(f"    {conclusion_preview}...")
    else:
        print("    (Non generee)")
    
    print("\n" + "="*70)
    
    return cv['validation_status']


# === EXECUTION DE LA VALIDATION ===
print("\n--- Generation du Rapport d'Analyse Validee ---")

# Chercher l'etat local_state defini par l'orchestration
if 'local_state' in dir() and local_state is not None:
    # Generate report
    final_report = generate_validated_analysis_report(local_state)
    
    # Display summary
    validation_status = display_validation_summary(final_report)
    
    # Export JSON
    output_path = "output/analysis_report.json"
    try:
        import os
        os.makedirs("output", exist_ok=True)
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(final_report, f, indent=2, ensure_ascii=False, default=str)
        print(f"\nRapport JSON exporte vers: {output_path}")
    except Exception as e:
        print(f"Erreur export JSON: {e}")
    
    # Afficher JSON complet
    print("\n--- RAPPORT JSON COMPLET ---")
    print(json.dumps(final_report, indent=2, ensure_ascii=False, default=str))
    
    # Final verdict
    print("\n--- VERDICT FINAL ---")
    if validation_status == "COMPLETE_VALIDATED":
        print("[SUCCESS] ANALYSE RHETORIQUE VALIDEE COMPLETE")
        print("  Tous les criteres de validation sont satisfaits.")
    elif validation_status == "PARTIAL_VALIDATED":
        print("[PARTIEL] Analyse partiellement validee")
        print("  Certaines etapes manquent pour une validation complete.")
    else:
        print(f"[{validation_status}] Analyse incomplete")
        print("  Verifiez les problemes detectes ci-dessus.")

else:
    print("[INFO] Etat d'analyse (local_state) non disponible.")
    print("  Executez d'abord les cellules precedentes pour lancer l'analyse.")
    print("  Ou l'analyse n'a pas ete executee (mode batch sans erreur?)")

### Interprétation du rapport de validation

**Statuts de validation possibles**:

| Statut | Score | Critères |
|--------|-------|----------|
| `COMPLETE_VALIDATED` | ≥80% | Tous les composants critiques présents |
| `PARTIAL_VALIDATED` | ≥50% | Analyse fonctionnelle mais incomplète |
| `MINIMAL` | ≥30% | Quelques éléments détectés |
| `INCOMPLETE` | <30% | Échec de l'analyse |

**Critères de validation**:

1. **ARGUMENTS_IDENTIFIED**: Au moins un argument détecté
2. **FALLACIES_ANALYZED**: Au moins un sophisme identifié
3. **BELIEF_SET_CREATED**: Au moins un Belief Set PL généré
4. **QUERIES_EXECUTED**: Au moins une requête logique lancée
5. **QUERIES_MEANINGFUL**: Résultat ACCEPTED ou REJECTED obtenu
6. **CONCLUSION_GENERATED**: Conclusion finale présente

**Exemple d'analyse réussie**:

```json
{
  "cross_validation": {
    "validation_status": "COMPLETE_VALIDATED",
    "confidence_score": 0.83,
    "checks_passed": [
      "ARGUMENTS_IDENTIFIED",
      "FALLACIES_ANALYZED",
      "BELIEF_SET_CREATED",
      "QUERIES_EXECUTED",
      "CONCLUSION_GENERATED"
    ],
    "issues": []
  }
}
```

> **Utilité du rapport JSON**: Le fichier `output/analysis_report.json` peut être utilisé pour:
> - Tests automatisés (CI/CD)
> - Métriques de qualité d'analyse
> - Comparaison entre versions du système
> - Documentation des résultats d'analyse

## 6. Résultats et Conclusion

Vérifiez les logs et l'état final JSON affichés par l'exécution précédente pour voir le résultat de l'analyse collaborative.

### Récapitulatif du workflow complet

Le notebook orchestrateur `Argument_Analysis_Executor.ipynb` implémente un pipeline d'analyse rhétorique en 7 étapes:

| Étape | Action | Durée typique |
|-------|--------|---------------|
| 1. Configuration | Chargement `.env`, variables d'environnement | <1s |
| 2. Interface/Batch | Configuration texte (UI ou batch) | Variable (0-60s) |
| 3. Chargement agents | Import des 4 notebooks modulaires | 5-10s |
| 4. Analyse informelle | Détection arguments + sophismes | 40-60s |
| 5. Formalisation PL | Création Belief Sets + requêtes | 30-50s |
| 6. Validation | Génération rapport JSON structuré | <1s |
| 7. Export | Sauvegarde `output/analysis_report.json` | <1s |

**Total (mode batch)**: ~120-150 secondes

### Méthodologie d'analyse rhétorique

Le système combine deux approches complémentaires:

**Analyse informelle (InformalAnalysisAgent)**:
- Utilise un LLM (GPT-4/Claude) pour comprendre le langage naturel
- Identifie arguments, prémisses, conclusions
- Détecte sophismes via taxonomie hiérarchique
- Output: Structure JSON décrivant la rhétorique

**Analyse formelle (PropositionalLogicAgent)**:
- Traduit les arguments en logique propositionnelle
- Crée des Belief Sets Tweety (bibliothèque Java)
- Exécute requêtes de cohérence et d'implication
- Output: Preuves formelles ou contre-exemples

> **Avantage de l'approche hybride**: La combinaison LLM + logique formelle permet à la fois la compréhension contextuelle (LLM) et la rigueur mathématique (logique).

## 7. Pistes d'Amélioration Futures

### Améliorations techniques

| Catégorie | Amélioration proposée | Impact |
|-----------|----------------------|--------|
| **Logique formelle** | Implémenter FOL, logique modale (Tweety) | Expressivité accrue |
| **Taxonomie sophismes** | Navigation profondeur variable, exemples | Précision détection |
| **Gestion erreurs** | Retry automatique, fallback gracieux | Robustesse production |
| **Performance** | Cache LLM, parallélisation agents | Réduction latence |
| **État RDF/KG** | Migration vers graphe sémantique (rdflib) | Requêtes complexes |

### Améliorations pédagogiques

**Visualisations interactives**:
- Graphe d'arguments (NetworkX + Plotly)
- Arbre taxonomique des sophismes
- Timeline de l'analyse (Gantt chart)

**Explications enrichies**:
- Définitions des sophismes avec exemples historiques
- Comparaison analyse informelle vs formelle
- Métriques de confiance par argument

**Interface utilisateur**:
- Dashboard Gradio/Streamlit pour upload de textes
- Annotation collaborative des arguments
- Export PDF avec rapport structuré

### Cas d'usage avancés

**Analyse de débats politiques**:
- Détection de manipulation rhétorique
- Comparaison positions de candidats
- Fact-checking automatisé (intégration sources externes)

**Éducation à la pensée critique**:
- Exercices interactifs sur textes annotés
- Quiz génératif sur détection de sophismes
- Feedback personnalisé sur essais d'étudiants

**Recherche académique**:
- Corpus analysis (analyse de milliers de textes)
- Évolution diachronique de l'argumentation
- Comparaison cultures/langues (multilingual)

> **Roadmap prioritaire**: 
> 1. Finaliser intégration Tweety (Query execution)
> 2. Implémenter visualisations graphiques
> 3. Ajouter interface Gradio
> 4. Publier package Python réutilisable